In [712]:
################### 买入后 上涨 大于30 的训练  
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [713]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [714]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal_u_30.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal_30.txt',header=None)

In [715]:
print(stock_lable.shape[0])

21349


In [716]:
################# 抽取出成功的标签，使成功与失败的比例是1:1.2
'''

'''
x_data = stock_data.values
x_data = x_data.reshape(-1,66)
#print(x_data.shape)
y_data = stock_lable.copy()
y_data['index'] = pd.DataFrame(np.arange(y_data.shape[0]))
y_data_1 = y_data[y_data[2]>0]
y_data_0 = y_data[y_data[2]==0]

y_data_0_sample = y_data_0.sample(n=int(y_data_1.shape[0]*1.2),replace=False)
y_train = pd.concat([y_data_1,y_data_0_sample])
index = y_train['index'].values
index = index.astype(int)
x_train = x_data[index.tolist()]
x_train = np.insert(x_train,0,1,axis=1)
x = x_train[::2,:]
y = y_train[2].values
y = y[::2].reshape(-1,1)

print('------------完成训练数据准备----------')

------------完成训练数据准备----------


In [717]:
print(x.shape[0])
print(y.shape[0])
print(x.shape)
print(y.shape)

1506
1506
(1506, 67)
(1506, 1)


In [721]:
########## 训练数据检查
#print(y_train.info())
for i in range(10):
    check_index = random.randint(0,x_train.shape[0]-1)
    index_a = y_train.iloc[check_index,3]
    print(check_index)
    print('输入的x',x_train[check_index])
    print('输入的y',y_train.iloc[check_index])
    print('原有的x',stock_data.iloc[index_a*6:index_a*6+5])
    print('原有的y',stock_lable.iloc[index_a])


2065
输入的x [ 1.      -1.57218 -1.18636 -1.21423 -1.65136  0.52902 -0.62558 -1.42622
 -1.1631   0.8165  -1.33599 -0.      -0.49411 -0.4975  -0.42234 -0.62638
  0.15901 -0.47476 -0.8335  -0.86544 -0.40825 -0.68163 -0.79057  0.04492
  0.42096 -0.26396  0.3986   0.32023 -0.43012 -0.09261 -0.33625 -0.40825
 -0.19086 -0.79057  0.58395 -0.72712 -0.10559  0.74026 -1.18884 -0.66724
  0.27783  0.09371 -1.63299 -0.02727 -0.79057  0.04492  1.56905  1.79495
  1.08192  1.3478   1.9168   0.90759  0.82134  0.8165   0.79069  0.79057
  1.3925   0.42096  0.21117  0.05694 -1.16722  0.28091  1.16691  1.44975
  0.8165   1.44505  1.58114]
输入的y 0            2131
1        20191014
2               0
index        3599
Name: 3599, dtype: int64
原有的x             0        1        2        3        4        5        6        7   \
21594 -1.57218 -1.18636 -1.21423 -1.65136  0.52902 -0.62558 -1.42622 -1.16310   
21595 -0.49411 -0.49750 -0.42234 -0.62638  0.15901 -0.47476 -0.83350 -0.86544   
21596  0.04492  0.42096 -0.

In [655]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [656]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 3000

In [702]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1_30.txt",delimiter=',')
w2 = np.loadtxt("init_w2_30.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.5
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-0.4217316  -0.44293066 -0.36803719  0.23397715  0.24579235]
 [ 0.94014674  0.34151493  0.68239923  0.63363951  0.64480186]
 [ 0.66675489  0.21078263 -0.27762248 -0.23546038  0.22058585]
 [-0.14745678  0.74765085 -0.30670244 -0.4276152  -0.6883972 ]
 [ 0.75024963  0.03228668  0.73878514 -0.72195593  0.09180386]
 [ 0.5619141  -0.25208774  0.54737452 -0.32875276 -0.65414385]
 [ 0.32650343 -0.07845116  0.96293421  0.44934147 -0.31247958]
 [-0.16989988  0.50315371  0.17627088  0.23458012  0.98308858]
 [ 0.89524505  0.72707015 -0.9665445   0.75737045 -0.44171008]
 [-0.50414152 -0.75807593 -0.59746724 -0.54084173  0.53522769]]
[[-0.73586041]
 [-0.03142738]
 [ 0.6299364 ]
 [-0.1921244 ]
 [ 0.28451736]
 [ 0.95065389]
 [-0.1815558 ]
 [ 0.3807564 ]
 [ 0.23147114]
 [ 0.58457722]]


In [288]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [703]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 2000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 2000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1_30.txt",w1,delimiter=',')
    np.savetxt("init_w2_30.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 1.2397426129903528
----------cost is ------------- 0.9404170450017986
----------cost is ------------- 1.0577243881925362
----------cost is ------------- 1.5023488912513503
----------cost is ------------- 1.686837780744934
----------cost is ------------- 1.241419188184316
----------cost is ------------- 1.4451359687202878
----------cost is ------------- 1.1098166256190451
----------cost is ------------- 1.2842114731562906
----------cost is ------------- 1.021566058652742
----------cost is ------------- 1.176022330279849
----------cost is ------------- 0.9515183582247402
----------cost is ------------- 1.0919840755242154
----------cost is ------------- 0.8951726060229662
----------cost is ------------- 1.0234346816876119
----------cost is ------------- 0.8486091409136063
----------cost is ------------- 0.9652987938655516
----------cost is ------------- 0.808843455128125
----------cost is ------------- 0.9146821852864229
----------cost is ------------- 0.77

----------cost is ------------- 0.25153406529940897
----------cost is ------------- 0.23930917060209636
----------cost is ------------- 0.24955863781923374
----------cost is ------------- 0.23745372665978437
----------cost is ------------- 0.2476193253290326
----------cost is ------------- 0.23563075536151193
----------cost is ------------- 0.24571456144900483
----------cost is ------------- 0.2338388685891867
----------cost is ------------- 0.24384282565053778
----------cost is ------------- 0.23207672200296295
----------cost is ------------- 0.24200264686962566
----------cost is ------------- 0.23034301757437603
----------cost is ------------- 0.24019260664784606
----------cost is ------------- 0.2286365057452523
----------cost is ------------- 0.23841134172814596
----------cost is ------------- 0.2269559871630773
----------cost is ------------- 0.23665754605961822
----------cost is ------------- 0.2253003139615576
----------cost is ------------- 0.2349299721899985
----------cost is 

----------cost is ------------- 0.13699676992780369
----------cost is ------------- 0.14238530047595016
----------cost is ------------- 0.13597277617423212
----------cost is ------------- 0.14129573525572764
----------cost is ------------- 0.13495103053141902
----------cost is ------------- 0.14020799847866958
----------cost is ------------- 0.13393147292996377
----------cost is ------------- 0.13912203425697653
----------cost is ------------- 0.1329140471622952
----------cost is ------------- 0.138037791344367
----------cost is ------------- 0.13189870087081437
----------cost is ------------- 0.13695522310805613
----------cost is ------------- 0.13088538552589193
----------cost is ------------- 0.1358742874882717
----------cost is ------------- 0.12987405639370697
----------cost is ------------- 0.1347949469455269
----------cost is ------------- 0.12886467249414102
----------cost is ------------- 0.13371716839613823
----------cost is ------------- 0.12785719654916294
----------cost is

----------cost is ------------- 0.06328969692267039
----------cost is ------------- 0.06240853212027223
----------cost is ------------- 0.06255658831830428
----------cost is ------------- 0.06171902306512742
----------cost is ------------- 0.061837129400015935
----------cost is ------------- 0.061041626123843305
----------cost is ------------- 0.06113155512998578
----------cost is ------------- 0.060376564524876365
----------cost is ------------- 0.06044007611568715
----------cost is ------------- 0.05972404140775812
----------cost is ------------- 0.05976287727497055
----------cost is ------------- 0.05908423859954496
----------cost is ------------- 0.05910011669402627
----------cost is ------------- 0.05845731554115793
----------cost is ------------- 0.058451924684852066
----------cost is ------------- 0.05784340836829157
----------cost is ------------- 0.057818403044237396
----------cost is ------------- 0.057242629147862864
----------cost is ------------- 0.05719962451185333
------

----------cost is ------------- 0.036099338211219696
----------cost is ------------- 0.03601170647374279
----------cost is ------------- 0.03592437622610752
----------cost is ------------- 0.035837346246782434
----------cost is ------------- 0.035750615334689075
----------cost is ------------- 0.035664182283673414
----------cost is ------------- 0.03557804590437097
----------cost is ------------- 0.03549220500478347
----------cost is ------------- 0.03540665840687213
----------cost is ------------- 0.035321404931860735
----------cost is ------------- 0.035236443412754476
----------cost is ------------- 0.03515177268327801
----------cost is ------------- 0.03506739158726555
----------cost is ------------- 0.03498329897037742
----------cost is ------------- 0.03489949368710224
----------cost is ------------- 0.03481597459458491
----------cost is ------------- 0.03473274055781718
----------cost is ------------- 0.03464979044506089
----------cost is ------------- 0.03456712313167215
------

...

----------cost is ------------- 0.011566073862956508
----------cost is ------------- 0.011551434936223222
----------cost is ------------- 0.011536827367213202
----------cost is ------------- 0.01152225106480803
----------cost is ------------- 0.011507705938214104
----------cost is ------------- 0.011493191896961304
----------cost is ------------- 0.011478708850901657
----------cost is ------------- 0.01146425671020803
----------cost is ------------- 0.011449835385372807
----------cost is ------------- 0.01143544478720655
----------cost is ------------- 0.01142108482683675
----------cost is ------------- 0.011406755415706465
----------cost is ------------- 0.011392456465573077
----------cost is ------------- 0.01137818788850698
----------cost is ------------- 0.011363949596890284
----------cost is ------------- 0.011349741503415587
----------cost is ------------- 0.011335563521084648
----------cost is ------------- 0.011321415563207166
----------cost is ------------- 0.01130729754339949

----------cost is ------------- 0.009611812888384669
----------cost is ------------- 0.009601137923235237
----------cost is ------------- 0.009590483324634913
----------cost is ------------- 0.00957984903914891
----------cost is ------------- 0.009569235013516983
----------cost is ------------- 0.009558641194652776
----------cost is ------------- 0.009548067529643144
----------cost is ------------- 0.0095375139657475
----------cost is ------------- 0.009526980450397142
----------cost is ------------- 0.00951646693119461
----------cost is ------------- 0.00950597335591303
----------cost is ------------- 0.009495499672495442
----------cost is ------------- 0.009485045829054182
----------cost is ------------- 0.009474611773870219
----------cost is ------------- 0.009464197455392507
----------cost is ------------- 0.009453802822237355
----------cost is ------------- 0.009443427823187774
----------cost is ------------- 0.009433072407192877
----------cost is ------------- 0.0094227365233672


----------cost is ------------- 0.008123185769430413
----------cost is ------------- 0.008115204873441839
----------cost is ------------- 0.008107237600276063
----------cost is ------------- 0.008099283917640011
----------cost is ------------- 0.008091343793336932
----------cost is ------------- 0.008083417195266066
----------cost is ------------- 0.00807550409142228
----------cost is ------------- 0.00806760444989575
----------cost is ------------- 0.008059718238871623
----------cost is ------------- 0.00805184542662967
----------cost is ------------- 0.008043985981543954
----------cost is ------------- 0.008036139872082513
----------cost is ------------- 0.00802830706680701
----------cost is ------------- 0.00802048753437241
----------cost is ------------- 0.00801268124352665
----------cost is ------------- 0.008004888163110313
----------cost is ------------- 0.007997108262056301
----------cost is ------------- 0.007989341509389512
----------cost is ------------- 0.007981587874226512

----------cost is ------------- 0.006987869041544296
----------cost is ------------- 0.006981732550005186
----------cost is ------------- 0.006975605502045392
----------cost is ------------- 0.006969487877338909
----------cost is ------------- 0.006963379655615176
----------cost is ------------- 0.006957280816658928
----------cost is ------------- 0.006951191340309966
----------cost is ------------- 0.006945111206463032
----------cost is ------------- 0.006939040395067588
----------cost is ------------- 0.006932978886127656
----------cost is ------------- 0.0069269266597016405
----------cost is ------------- 0.006920883695902145
----------cost is ------------- 0.006914849974895799
----------cost is ------------- 0.0069088254769030895
----------cost is ------------- 0.006902810182198164
----------cost is ------------- 0.006896804071108686
----------cost is ------------- 0.006890807124015636
----------cost is ------------- 0.006884819321353151
----------cost is ------------- 0.0068788406

----------cost is ------------- 0.006128453120877225
----------cost is ------------- 0.006123581492944903
----------cost is ------------- 0.006118716696603355
----------cost is ------------- 0.006113858718384981
----------cost is ------------- 0.0061090075448560245
----------cost is ------------- 0.006104163162616439
----------cost is ------------- 0.006099325558299799
----------cost is ------------- 0.006094494718573204
----------cost is ------------- 0.006089670630137184
----------cost is ------------- 0.006084853279725561
----------cost is ------------- 0.006080042654105393
----------cost is ------------- 0.006075238740076859
----------cost is ------------- 0.00607044152447314
----------cost is ------------- 0.006065650994160346
----------cost is ------------- 0.006060867136037402
----------cost is ------------- 0.006056089937035962
----------cost is ------------- 0.006051319384120284
----------cost is ------------- 0.006046555464287166
----------cost is ------------- 0.006041798164

In [735]:
### 先用训练集验证

index = random.randint(0,1505)
x_train_test = x[index]
x_train_test = x_train_test.reshape(1,-1)
#x_train_test = np.insert(x_train_test,0,1,axis=0)
y_test = y[index]
#y_t = y_train.iloc[index]
#print(y_t)
print('--序号--',index)
print('----输入-----',x_train_test.shape)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('-----输出-----',y_test,hpfs_output)

--序号-- 367
----输入----- (1, 67)
-----输出----- [1] [[0.99999977]]


In [765]:
x_test_1 = x_train[1::2,:]
y_1 = y_train[2].values
y_1 = y_1[1::2].reshape(-1,1)
index = random.randint(0,1505)
x_train_test = x_test_1[index]
x_train_test = x_train_test.reshape(1,-1)
print('--序号--',index)
print('----输入-----',x_train_test.shape)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('-----输出-----',y_1[index],hpfs_output)

--序号-- 513
----输入----- (1, 67)
-----输出----- [1] [[0.34144115]]


In [744]:
#index = random.randint(1501,2100)
error_data = pd.DataFrame(columns =['1','2','3'])
#index = 300
error_count = 0
#print(stock_data.head(20))
for index in range(0,19999):
    x_test = stock_data.loc[index*6:index*6+5].values
    x_test = x_test.reshape(1,-1)
    x_test = np.insert(x_test,0,1,axis=1)
    print('--序号--',index)
    #print('----输入-----',x_test)
    y_test = stock_lable.loc[index,2]
    hid_input = np.matmul(x_test,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=1)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    output = hpfs_output[0][0]
    y_value = y_test
    #print(output)
    if(output > 0.5 and y_value ==1) or (output<0.5 and y_value ==0):
        print('-----输出-----',y_test,hpfs_output)
    else:
        print('-----输出-----',y_test,hpfs_output)
        error_count = error_count + 1
        error_data.append(stock_lable.loc[index],ignore_index='True')
    #print('-----输出-----',y_test,hpfs_output)

--序号-- 0
-----输出----- 0 [[0.46685055]]
--序号-- 1
-----输出----- 0 [[0.7192749]]
--序号-- 2
-----输出----- 0 [[0.12974332]]
--序号-- 3
-----输出----- 0 [[0.04831457]]
--序号-- 4
-----输出----- 0 [[0.00378837]]
--序号-- 5
-----输出----- 0 [[0.60460298]]
--序号-- 6
-----输出----- 0 [[0.18362892]]
--序号-- 7
-----输出----- 0 [[1.92403585e-05]]
--序号-- 8
-----输出----- 0 [[0.31251279]]
--序号-- 9
-----输出----- 0 [[0.00017908]]
--序号-- 10
-----输出----- 0 [[0.9486484]]
--序号-- 11
-----输出----- 1 [[0.99812916]]
--序号-- 12
-----输出----- 0 [[0.17398734]]
--序号-- 13
-----输出----- 0 [[0.92422973]]
--序号-- 14
-----输出----- 0 [[0.47531097]]
--序号-- 15
-----输出----- 0 [[0.96020468]]
--序号-- 16
-----输出----- 0 [[0.99436784]]
--序号-- 17
-----输出----- 0 [[0.68654631]]
--序号-- 18
-----输出----- 0 [[0.00162335]]
--序号-- 19
-----输出----- 0 [[0.31594018]]
--序号-- 20
-----输出----- 0 [[0.98924421]]
--序号-- 21
-----输出----- 0 [[0.00458083]]
--序号-- 22
-----输出----- 0 [[0.00408311]]
--序号-- 23
-----输出----- 0 [[0.0001563]]
--序号-- 24
-----输出----- 0 [[0.58684366]]
--序号-- 25

--序号-- 303
-----输出----- 0 [[0.64048707]]
--序号-- 304
-----输出----- 0 [[0.9987274]]
--序号-- 305
-----输出----- 0 [[3.10102356e-06]]
--序号-- 306
-----输出----- 0 [[0.68257572]]
--序号-- 307
-----输出----- 0 [[0.3122385]]
--序号-- 308
-----输出----- 1 [[0.99611458]]
--序号-- 309
-----输出----- 0 [[0.53357446]]
--序号-- 310
-----输出----- 0 [[0.40274505]]
--序号-- 311
-----输出----- 1 [[0.02608121]]
--序号-- 312
-----输出----- 0 [[0.33244144]]
--序号-- 313
-----输出----- 0 [[0.98060354]]
--序号-- 314
-----输出----- 1 [[0.99704289]]
--序号-- 315
-----输出----- 0 [[0.00068254]]
--序号-- 316
-----输出----- 0 [[0.91978419]]
--序号-- 317
-----输出----- 0 [[0.00191399]]
--序号-- 318
-----输出----- 0 [[0.86464058]]
--序号-- 319
-----输出----- 0 [[0.0007307]]
--序号-- 320
-----输出----- 0 [[0.67134395]]
--序号-- 321
-----输出----- 0 [[5.44902239e-05]]
--序号-- 322
-----输出----- 0 [[0.96234319]]
--序号-- 323
-----输出----- 0 [[0.07323482]]
--序号-- 324
-----输出----- 0 [[0.99582372]]
--序号-- 325
-----输出----- 0 [[0.00016771]]
--序号-- 326
-----输出----- 0 [[0.01098393]]
--序号-- 327


-----输出----- 0 [[0.91074916]]
--序号-- 609
-----输出----- 0 [[0.15251866]]
--序号-- 610
-----输出----- 0 [[0.07591542]]
--序号-- 611
-----输出----- 0 [[9.57016627e-05]]
--序号-- 612
-----输出----- 1 [[0.00315326]]
--序号-- 613
-----输出----- 0 [[0.08013777]]
--序号-- 614
-----输出----- 0 [[2.00626648e-05]]
--序号-- 615
-----输出----- 0 [[0.92299778]]
--序号-- 616
-----输出----- 0 [[0.99823376]]
--序号-- 617
-----输出----- 0 [[2.20245317e-05]]
--序号-- 618
-----输出----- 0 [[0.99998259]]
--序号-- 619
-----输出----- 0 [[0.07968192]]
--序号-- 620
-----输出----- 0 [[0.01958773]]
--序号-- 621
-----输出----- 0 [[0.00018919]]
--序号-- 622
-----输出----- 0 [[0.97263776]]
--序号-- 623
-----输出----- 0 [[0.00036335]]
--序号-- 624
-----输出----- 0 [[5.78391772e-06]]
--序号-- 625
-----输出----- 0 [[1.69855542e-07]]
--序号-- 626
-----输出----- 0 [[0.99998887]]
--序号-- 627
-----输出----- 0 [[0.87752423]]
--序号-- 628
-----输出----- 0 [[0.76936921]]
--序号-- 629
-----输出----- 0 [[0.98745812]]
--序号-- 630
-----输出----- 0 [[0.4700429]]
--序号-- 631
-----输出----- 0 [[3.34768418e-05]]
--序号

--序号-- 811
-----输出----- 0 [[0.00079469]]
--序号-- 812
-----输出----- 0 [[0.70280793]]
--序号-- 813
-----输出----- 0 [[0.95586814]]
--序号-- 814
-----输出----- 0 [[0.83418117]]
--序号-- 815
-----输出----- 0 [[0.00012599]]
--序号-- 816
-----输出----- 0 [[0.99289068]]
--序号-- 817
-----输出----- 0 [[0.38631046]]
--序号-- 818
-----输出----- 0 [[0.00032257]]
--序号-- 819
-----输出----- 0 [[0.0508125]]
--序号-- 820
-----输出----- 0 [[0.93061182]]
--序号-- 821
-----输出----- 0 [[0.06590583]]
--序号-- 822
-----输出----- 0 [[0.04382587]]
--序号-- 823
-----输出----- 0 [[0.98787978]]
--序号-- 824
-----输出----- 0 [[0.63249665]]
--序号-- 825
-----输出----- 0 [[0.00740877]]
--序号-- 826
-----输出----- 0 [[8.84431844e-05]]
--序号-- 827
-----输出----- 0 [[0.61428077]]
--序号-- 828
-----输出----- 0 [[4.25887106e-05]]
--序号-- 829
-----输出----- 0 [[0.01220958]]
--序号-- 830
-----输出----- 0 [[0.0652044]]
--序号-- 831
-----输出----- 0 [[0.00018383]]
--序号-- 832
-----输出----- 0 [[0.057445]]
--序号-- 833
-----输出----- 0 [[0.99894946]]
--序号-- 834
-----输出----- 0 [[0.99666045]]
--序号-- 835
-

-----输出----- 0 [[3.8079154e-06]]
--序号-- 1090
-----输出----- 0 [[1.45432174e-05]]
--序号-- 1091
-----输出----- 1 [[0.96500727]]
--序号-- 1092
-----输出----- 0 [[0.00094384]]
--序号-- 1093
-----输出----- 0 [[2.13869742e-07]]
--序号-- 1094
-----输出----- 0 [[0.95211546]]
--序号-- 1095
-----输出----- 0 [[0.00148959]]
--序号-- 1096
-----输出----- 1 [[0.99972479]]
--序号-- 1097
-----输出----- 0 [[4.43164341e-06]]
--序号-- 1098
-----输出----- 0 [[0.00028016]]
--序号-- 1099
-----输出----- 0 [[0.72958193]]
--序号-- 1100
-----输出----- 0 [[0.95478339]]
--序号-- 1101
-----输出----- 1 [[0.98860151]]
--序号-- 1102
-----输出----- 0 [[0.88354588]]
--序号-- 1103
-----输出----- 0 [[2.46302641e-08]]
--序号-- 1104
-----输出----- 0 [[0.93887123]]
--序号-- 1105
-----输出----- 0 [[0.00101888]]
--序号-- 1106
-----输出----- 1 [[0.99989458]]
--序号-- 1107
-----输出----- 0 [[0.00256528]]
--序号-- 1108
-----输出----- 0 [[3.44479853e-05]]
--序号-- 1109
-----输出----- 0 [[0.01795565]]
--序号-- 1110
-----输出----- 0 [[0.98714455]]
--序号-- 1111
-----输出----- 0 [[0.00753795]]
--序号-- 1112
-----输出----

...

--序号-- 18918
-----输出----- 0 [[0.99811392]]
--序号-- 18919
-----输出----- 0 [[4.75653909e-05]]
--序号-- 18920
-----输出----- 0 [[0.05289544]]
--序号-- 18921
-----输出----- 0 [[0.00075455]]
--序号-- 18922
-----输出----- 0 [[0.00124774]]
--序号-- 18923
-----输出----- 0 [[0.06304057]]
--序号-- 18924
-----输出----- 0 [[0.61004633]]
--序号-- 18925
-----输出----- 0 [[0.9902679]]
--序号-- 18926
-----输出----- 0 [[0.14557035]]
--序号-- 18927
-----输出----- 0 [[0.0009564]]
--序号-- 18928
-----输出----- 0 [[7.43283329e-05]]
--序号-- 18929
-----输出----- 0 [[0.03712385]]
--序号-- 18930
-----输出----- 0 [[0.04283672]]
--序号-- 18931
-----输出----- 0 [[0.6828498]]
--序号-- 18932
-----输出----- 0 [[0.99626385]]
--序号-- 18933
-----输出----- 0 [[0.00125382]]
--序号-- 18934
-----输出----- 0 [[0.00315463]]
--序号-- 18935
-----输出----- 0 [[0.52849584]]
--序号-- 18936
-----输出----- 0 [[1.14648828e-05]]
--序号-- 18937
-----输出----- 0 [[0.49968058]]
--序号-- 18938
-----输出----- 0 [[7.07580855e-05]]
--序号-- 18939
-----输出----- 0 [[0.83836567]]
--序号-- 18940
-----输出----- 0 [[0.03038924]

--序号-- 19218
-----输出----- 0 [[0.02137899]]
--序号-- 19219
-----输出----- 0 [[0.503486]]
--序号-- 19220
-----输出----- 0 [[0.70184325]]
--序号-- 19221
-----输出----- 0 [[0.00278477]]
--序号-- 19222
-----输出----- 0 [[0.85538395]]
--序号-- 19223
-----输出----- 1 [[0.93248915]]
--序号-- 19224
-----输出----- 0 [[0.08402309]]
--序号-- 19225
-----输出----- 0 [[0.00080488]]
--序号-- 19226
-----输出----- 0 [[0.97782991]]
--序号-- 19227
-----输出----- 1 [[0.98275545]]
--序号-- 19228
-----输出----- 0 [[0.02098193]]
--序号-- 19229
-----输出----- 0 [[0.01095136]]
--序号-- 19230
-----输出----- 0 [[0.1108591]]
--序号-- 19231
-----输出----- 0 [[0.99962316]]
--序号-- 19232
-----输出----- 0 [[0.59504198]]
--序号-- 19233
-----输出----- 0 [[0.023183]]
--序号-- 19234
-----输出----- 0 [[0.03059483]]
--序号-- 19235
-----输出----- 1 [[0.97623319]]
--序号-- 19236
-----输出----- 0 [[0.13010473]]
--序号-- 19237
-----输出----- 0 [[0.9967783]]
--序号-- 19238
-----输出----- 0 [[0.27634376]]
--序号-- 19239
-----输出----- 0 [[0.99980411]]
--序号-- 19240
-----输出----- 0 [[0.01602203]]
--序号-- 19241
----

--序号-- 19470
-----输出----- 0 [[0.99430061]]
--序号-- 19471
-----输出----- 0 [[0.98971107]]
--序号-- 19472
-----输出----- 1 [[0.96095479]]
--序号-- 19473
-----输出----- 0 [[0.99937964]]
--序号-- 19474
-----输出----- 0 [[0.71630901]]
--序号-- 19475
-----输出----- 0 [[1.64776516e-05]]
--序号-- 19476
-----输出----- 0 [[0.99764618]]
--序号-- 19477
-----输出----- 0 [[0.04846826]]
--序号-- 19478
-----输出----- 0 [[0.99964685]]
--序号-- 19479
-----输出----- 0 [[2.84627509e-06]]
--序号-- 19480
-----输出----- 0 [[0.77823014]]
--序号-- 19481
-----输出----- 0 [[0.120895]]
--序号-- 19482
-----输出----- 0 [[0.04515119]]
--序号-- 19483
-----输出----- 0 [[2.00630116e-06]]
--序号-- 19484
-----输出----- 0 [[0.00052414]]
--序号-- 19485
-----输出----- 0 [[0.10158949]]
--序号-- 19486
-----输出----- 0 [[2.48929425e-09]]
--序号-- 19487
-----输出----- 0 [[0.0436264]]
--序号-- 19488
-----输出----- 0 [[0.00664144]]
--序号-- 19489
-----输出----- 0 [[0.00526025]]
--序号-- 19490
-----输出----- 0 [[7.72558106e-06]]
--序号-- 19491
-----输出----- 0 [[0.98988358]]
--序号-- 19492
-----输出----- 0 [[2.41720

-----输出----- 0 [[6.58078696e-08]]
--序号-- 19718
-----输出----- 0 [[2.45510934e-05]]
--序号-- 19719
-----输出----- 0 [[0.61568587]]
--序号-- 19720
-----输出----- 0 [[3.045438e-05]]
--序号-- 19721
-----输出----- 0 [[0.00133806]]
--序号-- 19722
-----输出----- 0 [[6.50780535e-05]]
--序号-- 19723
-----输出----- 0 [[0.98777802]]
--序号-- 19724
-----输出----- 0 [[0.01223751]]
--序号-- 19725
-----输出----- 0 [[0.99992454]]
--序号-- 19726
-----输出----- 0 [[0.00026718]]
--序号-- 19727
-----输出----- 0 [[6.22017927e-05]]
--序号-- 19728
-----输出----- 0 [[0.97016845]]
--序号-- 19729
-----输出----- 0 [[0.04149133]]
--序号-- 19730
-----输出----- 0 [[0.27554305]]
--序号-- 19731
-----输出----- 0 [[0.00048083]]
--序号-- 19732
-----输出----- 0 [[0.20312432]]
--序号-- 19733
-----输出----- 0 [[0.0117199]]
--序号-- 19734
-----输出----- 0 [[0.13881599]]
--序号-- 19735
-----输出----- 0 [[0.02266516]]
--序号-- 19736
-----输出----- 0 [[5.55559039e-06]]
--序号-- 19737
-----输出----- 0 [[0.05118192]]
--序号-- 19738
-----输出----- 0 [[0.02407744]]
--序号-- 19739
-----输出----- 0 [[1.70612746e-09]]

-----输出----- 1 [[0.99998978]]
--序号-- 19960
-----输出----- 0 [[0.55013791]]
--序号-- 19961
-----输出----- 0 [[0.43837605]]
--序号-- 19962
-----输出----- 0 [[0.00975963]]
--序号-- 19963
-----输出----- 0 [[0.00026046]]
--序号-- 19964
-----输出----- 0 [[7.07276049e-06]]
--序号-- 19965
-----输出----- 0 [[0.54374416]]
--序号-- 19966
-----输出----- 0 [[0.00946465]]
--序号-- 19967
-----输出----- 0 [[0.99864764]]
--序号-- 19968
-----输出----- 0 [[0.00205461]]
--序号-- 19969
-----输出----- 0 [[6.91170094e-05]]
--序号-- 19970
-----输出----- 0 [[0.68671657]]
--序号-- 19971
-----输出----- 0 [[0.76431427]]
--序号-- 19972
-----输出----- 1 [[0.97365879]]
--序号-- 19973
-----输出----- 0 [[6.89714967e-06]]
--序号-- 19974
-----输出----- 1 [[0.11142997]]
--序号-- 19975
-----输出----- 1 [[0.9917879]]
--序号-- 19976
-----输出----- 0 [[0.01995338]]
--序号-- 19977
-----输出----- 0 [[0.50744301]]
--序号-- 19978
-----输出----- 0 [[0.00231465]]
--序号-- 19979
-----输出----- 0 [[0.00110441]]
--序号-- 19980
-----输出----- 0 [[0.96263429]]
--序号-- 19981
-----输出----- 0 [[0.05243311]]
--序号-- 19982


In [452]:
print(error_data.shape)

(0, 3)


In [453]:
print(error_data.values)

[]


In [745]:
error_count

5669